In [115]:
import pandas as pd

# Import Files

In [116]:
customers=pd.read_csv("customers.csv")
order_items=pd.read_csv("order_items.csv")
orders=pd.read_csv("orders.csv")
payments=pd.read_csv("payments.csv")
products=pd.read_csv("products.csv")
sellers=pd.read_csv("sellers.csv")

/tmp/ipython-input-3793964878.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  order_items=pd.read_csv("order_items.csv")


#Basic Problems

##1) List all unique cities where customers are located

In [117]:
unique_cities = customers['customer_city'].dropna().unique()
sorted(unique_cities)

['abadia dos dourados',
 'abaete',
 'abaetetuba',
 'abaiara',
 'abatia',
 'abdon batista',
 'abrantes',
 'abre campo',
 'abreu e lima',
 'acailandia',
 'acarau',
 'acopiara',
 'acreuna',
 'acu',
 'adamantina',
 'adolfo',
 'adustina',
 'afogados da ingazeira',
 'afonso claudio',
 'agronomica',
 'agua boa',
 'agua branca',
 'agua doce do norte',
 'agua fria de goias',
 'agua santa',
 'aguai',
 'aguas belas',
 'aguas da prata',
 'aguas de lindoia',
 'aguas de sao pedro',
 'aguas formosas',
 'aguas frias',
 'aguas lindas de goias',
 'aguas mornas',
 'agudo',
 'agudos',
 'aimores',
 'aiuruoca',
 'ajuricaba',
 'alagoa',
 'alagoa grande',
 'alagoinhas',
 'alambari',
 'alcinopolis',
 'alcobaca',
 'alegre',
 'alegrete',
 'alem paraiba',
 'alexandria',
 'alexania',
 'alfenas',
 'alfredo chaves',
 'alfredo marcondes',
 'alfredo vasconcelos',
 'alfredo wagner',
 'alhandra',
 'almenara',
 'almino afonso',
 'almirante tamandare',
 'almirante tamandare do sul',
 'alpercata',
 'alpinopolis',
 'alta fl

In [118]:
len(unique_cities)

2703

##2) Count number of orders placed in 2017

In [119]:
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])

orders_2017 = orders[orders['order_purchase_timestamp'].dt.year == 2017]
len(orders_2017)

28140

##3) Total sales per category

In [120]:
merged = order_items.merge(products, on='product_id')

sales_per_category = (
    merged.groupby('product_category')['price']
    .sum()
    .sort_values(ascending=False)
)

sales_per_category

,price
product_category,
HEALTH BEAUTY,1258681.34
Watches present,1205005.68
bed table bath,1036988.68
sport leisure,988048.97
computer accessories,911954.32
...,...
flowers,1110.04
House Comfort 2,760.27
cds music dvds,730.00


##4) Percentage of orders paid in installments

In [121]:
installment_percentage = (
    (payments[payments['payment_installments'] > 1].shape[0]
     / payments.shape[0]) * 100
)

installment_percentage

49.41763086460158

##5) Number of customers from each state

In [122]:
customers_by_state = (
    customers.groupby('customer_state')
    .size()
    .sort_values(ascending=False)
)

customers_by_state

,0
customer_state,
SP,12522
RJ,3971
MG,3507
RS,1629
PR,1522
SC,1056
BA,980
DF,643
ES,620


In [123]:
len(customers_by_state)

27

#Intermediate Problems

##1) Orders per month in 2018

In [124]:
orders_2018 = orders[orders['order_purchase_timestamp'].dt.year == 2018]

orders_2018['month'] = orders_2018['order_purchase_timestamp'].dt.month

orders_2018.groupby('month').size().sort_index()

/tmp/ipython-input-2783707705.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  orders_2018['month'] = orders_2018['order_purchase_timestamp'].dt.month


,0
month,
1,4566
2,4235
3,4490
4,4347
5,4320
6,3842
7,3951
8,4125
9,12


##2) Average number of products per order grouped by customer city

In [125]:
merged = orders.merge(customers, on='customer_id') \
               .merge(order_items, on='order_id')

order_counts = merged.groupby(['order_id', 'customer_city']) \
                     .size().reset_index(name='product_count')

order_counts.groupby('customer_city')['product_count'].mean()

,product_count
customer_city,
abaete,1.0
abaetetuba,1.0
abaiara,1.0
abatia,1.0
abrantes,1.0
...,...
xavantina,1.0
xaxim,1.0
xinguara,1.0


##3) % revenue contribution by each category

In [126]:
base = (
    order_items[['order_id','product_id','price']]
    .merge(products[['product_id','product_category']],
           on='product_id',
           how='inner')
)

category_sales = (
    base
    .groupby('product_category')['price']
    .sum()
)

total_sales = base['price'].sum()

percentage = (category_sales * 100.0 / total_sales).round(2)

percentage = percentage.sort_values(ascending=False)

percentage

,price
product_category,
HEALTH BEAUTY,9.26
Watches present,8.87
bed table bath,7.63
sport leisure,7.27
computer accessories,6.71
...,...
Hygiene diapers,0.01
flowers,0.01
PC Gamer,0.01


##4) Correlation between price and purchase frequency

In [127]:
product_stats = (
    order_items
    .groupby('product_id')
    .agg(
        avg_price=('price', 'mean'),
        purchase_count=('order_id', 'count')
    )
    .reset_index()
)

product_stats

,product_id,avg_price,purchase_count
0,00066f42aeeb9f3007548bb9d3f33c38,101.65,1
1,00088930e925c41fd95ebfe695fd2655,129.90,1
2,0009406fd7479715e4bef61dd91f2462,229.00,1
3,000b8f95fcb9e0096488278317764d19,58.90,2
4,000d9be29b5207b54e86aa1b1ac54872,199.00,1
...,...,...,...
36041,fff9553ac224cec9d15d49f5a263411f,32.00,1
36042,fffd5413c0700ac820c7069d66d98c89,NaN,0
36043,fffdb2d0ec8d6a61f0a0a0db3f25b441,33.99,5
36044,fffe9eeff12fcbd74a2f2b007dde0c58,249.99,1


In [128]:
correlation = product_stats['avg_price'].corr(product_stats['purchase_count'])

correlation

np.float64(-0.032139862680945167)

## 5) Total revenue per seller + ranking

In [129]:
seller_revenue = order_items.groupby('product_id')['price'].sum()

seller_rank = seller_revenue.sort_values(ascending=False) \
                            .rank(method='dense', ascending=False)

pd.DataFrame({
    'total_revenue': seller_revenue,
    'rank': seller_rank
}).sort_values('rank')

,total_revenue,rank
product_id,,
bb50f2e236e5eea0100680137654686c,63885.00,1.0
6cdd53843498f92890544667809f1595,54730.20,2.0
d6160fb7873f184099d9bc95e30376af,48899.34,3.0
d1c427060a0f73f6b889a5c7c61f2ac4,47214.51,4.0
99a4788cb24856965c36a24e339b6058,43025.56,5.0
...,...,...
004c9cd9d87a3c30c522c48c4fc07416,0.00,10311.0
003554e2dce176b5555353e4f3555ac8,0.00,10311.0
ffff564a4f9085cd26170f4732393726,0.00,10311.0


#Advance Problems

##1) Moving average of order values per customer

In [130]:
orders['order_purchase_timestamp'] = pd.to_datetime(
    orders['order_purchase_timestamp']
)

order_values = (
    orders
    .merge(order_items, on='order_id', how='inner')
    .groupby(['order_id','customer_id','order_purchase_timestamp'], as_index=False)
    .agg(order_value=('price','sum'))
)

order_values = order_values.sort_values(
    ['customer_id','order_purchase_timestamp']
)

order_values['moving_avg_order_value'] = (
    order_values
    .groupby('customer_id')['order_value']
    .rolling(window=3, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)

order_values

,order_id,customer_id,order_purchase_timestamp,order_value,moving_avg_order_value
39451,a44895d095d7e0702b6a162fa2dbeced,000161a058600d5901f007fab4c27140,2017-07-16 09:40:32,54.90,54.90
21198,5825ce2e88d5346438686b0bba99e5ee,0002414f95344307404f0ace7a26f1d5,2017-08-16 13:09:20,149.90,149.90
2611,0ab7fb08086d4af9141453c91878ed7a,000379cdec625522490c315e70c7a9fb,2018-04-02 13:42:17,93.00,93.00
1937,07f6c3baf9ac86865b60f640c4f923c6,000419c5494106c306a97b5635748086,2018-03-02 17:47:40,34.30,34.30
33735,8c3d752c5c02227878fae49aeaddbfd7,00046a560d407e99b969756e0b10f282,2017-12-18 11:08:30,120.90,120.90
...,...,...,...,...,...
16233,43f08913407cac4e74a898d968e58c1a,fff7466a253c0e59499ea943462c10f9,2018-02-14 13:22:12,152.99,152.99
28005,745865dd655cbb9dc84b1ee5a872fbf9,fffa0238b217e18a8adeeda0669923a3,2017-09-11 16:02:18,35.00,35.00
23590,620ddc9fbe3e445676a8e71d4a830821,fffcb937e9dd47a13f05ecb8290f4d3e,2018-03-17 00:55:27,78.00,78.00
31144,814d6a3a7c0b32b2ad929ac6328124e9,fffecc9f79fd8c764f843e9951b11341,2018-03-29 16:59:26,54.90,54.90


##2) Cumulative monthly sales per year

In [131]:
orders['order_purchase_timestamp'] = pd.to_datetime(
    orders['order_purchase_timestamp']
)

sales_data = orders.merge(order_items, on='order_id', how='inner')

sales_data['year'] = sales_data['order_purchase_timestamp'].dt.year
sales_data['month'] = sales_data['order_purchase_timestamp'].dt.month

monthly_sales = (
    sales_data
    .groupby(['year','month'], as_index=False)
    .agg(monthly_sales=('price','sum'))
)

monthly_sales = monthly_sales.sort_values(['year','month'])

monthly_sales['cumulative_sales'] = (
    monthly_sales
    .groupby('year')['monthly_sales']
    .cumsum()
)

monthly_sales


,year,month,monthly_sales,cumulative_sales
0,2016,9,267.36,267.36
1,2016,10,31068.57,31335.93
2,2016,12,10.90,31346.83
3,2017,1,76471.68,76471.68
4,2017,2,148670.10,225141.78
5,2017,3,236080.32,461222.10
6,2017,4,223596.30,684818.40
7,2017,5,329011.47,1013829.87
8,2017,6,272214.02,1286043.89
9,2017,7,316300.89,1602344.78


##3) Year-over-Year Growth Rate

In [132]:
orders['order_purchase_timestamp'] = pd.to_datetime(
    orders['order_purchase_timestamp']
)

sales_data = orders.merge(order_items, on='order_id', how='inner')

sales_data['year'] = sales_data['order_purchase_timestamp'].dt.year

yearly_sales = (
    sales_data
    .groupby('year', as_index=False)
    .agg(total_sales=('price','sum'))
)

yearly_sales = yearly_sales.sort_values('year')

yearly_sales['previous_year'] = yearly_sales['total_sales'].shift(1)

yearly_sales['yoy_growth_percentage'] = (
    (yearly_sales['total_sales'] - yearly_sales['previous_year'])
    * 100.0
    / yearly_sales['previous_year']
).round(2)

yearly_sales

,year,total_sales,previous_year,yoy_growth_percentage
0,2016,31346.83,NaN,NaN
1,2017,3862173.32,31346.83,12220.78
2,2018,4624962.87,3862173.32,19.75


##4) 6-Month Customer Retention Rate

In [133]:
orders['order_purchase_timestamp'] = pd.to_datetime(
    orders['order_purchase_timestamp']
)

orders_customers = orders.merge(customers, on='customer_id', how='inner')

first_orders = (
    orders_customers
    .groupby('customer_unique_id', as_index=False)
    .agg(first_order_date=('order_purchase_timestamp','min'))
)

merged_ret = orders_customers.merge(
    first_orders,
    on='customer_unique_id',
    how='inner'
)

repeat_customers = merged_ret[
    (merged_ret['order_purchase_timestamp'] > merged_ret['first_order_date']) &
    (merged_ret['order_purchase_timestamp'] <=
     merged_ret['first_order_date'] + pd.DateOffset(months=6))
]

retention_rate = round(
    repeat_customers['customer_unique_id'].nunique()
    * 100.0
    / first_orders['customer_unique_id'].nunique(),
    2
)

retention_rate

0.48

##5) Top 3 customers by spending each year

In [134]:
merged_sql_logic = (
    orders
    .merge(customers[['customer_id', 'customer_unique_id']],
           on='customer_id', how='inner')
    .merge(order_items[['order_id', 'price']],
           on='order_id', how='inner')
)

merged_sql_logic['year'] = merged_sql_logic['order_purchase_timestamp'].dt.year

yearly_customer_spend = (
    merged_sql_logic
    .groupby(['year', 'customer_unique_id'])['price']
    .sum()
    .reset_index(name='total_spent')
)

yearly_customer_spend['rank_in_year'] = (
    yearly_customer_spend
    .groupby('year')['total_spent']
    .rank(method='dense', ascending=False)
)

top_3_customers = yearly_customer_spend[
    yearly_customer_spend['rank_in_year'] <= 3
].sort_values(['year', 'rank_in_year'])

top_3_customers

,year,customer_unique_id,total_spent,rank_in_year
51,2016,ba1a0b654ccab39f3691fbc1b35c2499,649.17,1.0
13,2016,3b2aa495b3557acad84b380301a9cc94,629.90,2.0
15,2016,3b6d8779a13d0e022d93c46e814de0fa,629.90,2.0
18,2016,45d9410bc1f3c6e36dea41d8f823ada6,599.00,3.0
390,2017,0a0a92112bd4c708ca5fde585afaa872,13440.00,1.0
7271,2017,dc4802a71eae9be1dd28f5d788ceb526,6735.00,2.0
8392,2017,ff4159b92c40ebe40454e3e6a7c35ed6,6499.00,3.0
11135,2018,459bef486812aa25204be022145caa62,6729.00,1.0
18334,2018,fa562ef24d41361e476e748681810e1e,4099.99,2.0
11161,2018,46450c74a0d8c5ca9395da1daac6c120,3109.99,3.0
